### fast local attention + crf 

#### batch size 128, learning rate = 0.001, windows size 5, num_gpus = 6


In [1]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.WARN)
import pickle
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
import os
from tensorflow.python.client import device_lib
from collections import Counter
import time

VERY_BIG_NUMBER = 1e30

In [2]:
f = open('../../Glove/word_embedding_glove', 'rb')
word_embedding = pickle.load(f)
f.close()

word_embedding = word_embedding[: len(word_embedding)-1]

f = open('../../Glove/vocab_glove', 'rb')
vocab = pickle.load(f)
f.close()

word2id = dict((w, i) for i,w in enumerate(vocab))
id2word = dict((i, w) for i,w in enumerate(vocab))

unknown_token = "UNKNOWN_TOKEN"

# Model Description
model_name = 'model-aw-lex-local-att-fast-v3-1'
model_dir = '../output/all-word/' + model_name
save_dir = os.path.join(model_dir, "save/")
log_dir = os.path.join(model_dir, "log")

if not os.path.exists(model_dir):
    os.mkdir(model_dir)
if not os.path.exists(save_dir):
    os.mkdir(save_dir)
if not os.path.exists(log_dir):
    os.mkdir(log_dir)

with open('../../../dataset/train_val_data_fine/all_word_lex','rb') as f:
    train_data, val_data = pickle.load(f)    
    
# Parameters
mode = 'train'
num_senses = 45
num_pos = 12
batch_size = 128
vocab_size = len(vocab)
unk_vocab_size = 1
word_emb_size = len(word_embedding[0])
max_sent_size = 200
hidden_size = 256
num_filter = 256
window_size = 5
kernel_size = 5
keep_prob = 0.3
l2_lambda = 0.001
init_lr = 0.001
decay_steps = 500
decay_rate = 0.99
clip_norm = 1
clipping = True
crf_lambda = 0.05
moving_avg_deacy = 0.999
num_gpus = 6
width = int(window_size/2)

In [3]:
def average_gradients(tower_grads):
    average_grads = []
    for grad_and_vars in zip(*tower_grads):
        # Note that each grad_and_vars looks like the following:
        #   ((grad0_gpu0, var0_gpu0), ... , (grad0_gpuN, var0_gpuN))
        grads = []
        for g, _ in grad_and_vars:
            # Add 0 dimension to the gradients to represent the tower.
            expanded_g = tf.expand_dims(g, 0)

            # Append on a 'tower' dimension which we will average over below.
            grads.append(expanded_g)

        # Average over the 'tower' dimension.
        grad = tf.concat(grads, 0)
        grad = tf.reduce_mean(grad, axis=0)

        # Keep in mind that the Variables are redundant because they are shared
        # across towers. So .. we will just return the first tower's pointer to
        # the Variable.
        v = grad_and_vars[0][1]
        grad_and_var = (grad, v)
        average_grads.append(grad_and_var)
    return average_grads

In [4]:
# MODEL
device_num = 0
tower_grads = []
losses = []
predictions = []
predictions_pos = []
total_trans_params = []

x = tf.placeholder('int32', [num_gpus, batch_size, max_sent_size], name="x")
y = tf.placeholder('int32', [num_gpus, batch_size, max_sent_size], name="y")
y_pos = tf.placeholder('int32', [num_gpus, batch_size, max_sent_size], name="y")
x_mask  = tf.placeholder('bool', [num_gpus, batch_size, max_sent_size], name='x_mask') 
sense_mask  = tf.placeholder('bool', [num_gpus, batch_size, max_sent_size], name='sense_mask')
is_train = tf.placeholder('bool', [], name='is_train')
word_emb_mat = tf.placeholder('float', [None, word_emb_size], name='emb_mat')
input_keep_prob = tf.cond(is_train,lambda:keep_prob, lambda:tf.constant(1.0))
pretrain = tf.placeholder('bool', [], name="pretrain")

global_step = tf.Variable(0, trainable=False, name="global_step")
learning_rate = tf.train.exponential_decay(init_lr, global_step, decay_steps, decay_rate, staircase=True)
summaries = []

def global_attention(input_x, input_mask, W_att):
    h_masked = tf.boolean_mask(input_x, input_mask)
    h_tanh = tf.tanh(h_masked)
    u = tf.matmul(h_tanh, W_att)
    a = tf.nn.softmax(u)
    c = tf.reduce_sum(tf.multiply(h_masked, a), axis=0)  
    return c

with tf.variable_scope("word_embedding"):
    unk_word_emb_mat = tf.get_variable("word_emb_mat", dtype='float', shape=[unk_vocab_size, word_emb_size], initializer=tf.contrib.layers.xavier_initializer(uniform=True, seed=0, dtype=tf.float32))
    final_word_emb_mat = tf.concat([word_emb_mat, unk_word_emb_mat], 0)

with tf.variable_scope(tf.get_variable_scope()):
    for gpu_idx in range(num_gpus):
        if gpu_idx>=3:
            device_num = 1
        with tf.name_scope("model_{}".format(gpu_idx)) as scope, tf.device('/gpu:%d' % device_num):

            if gpu_idx > 0:
                    tf.get_variable_scope().reuse_variables()

            with tf.name_scope("word"):
                Wx = tf.nn.embedding_lookup(final_word_emb_mat, x[gpu_idx])  

            float_x_mask = tf.cast(x_mask[gpu_idx], 'float')
            x_len = tf.reduce_sum(tf.cast(x_mask[gpu_idx], 'int32'), axis=1)
            
            with tf.variable_scope("lstm1"):
                cell_fw1 = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)
                cell_bw1 = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)

                d_cell_fw1 = tf.contrib.rnn.DropoutWrapper(cell_fw1, input_keep_prob=input_keep_prob)
                d_cell_bw1 = tf.contrib.rnn.DropoutWrapper(cell_bw1, input_keep_prob=input_keep_prob)

                (fw_h1, bw_h1), _ = tf.nn.bidirectional_dynamic_rnn(d_cell_fw1, d_cell_bw1, Wx, sequence_length=x_len, dtype='float', scope='lstm1')
                h1 = tf.concat([fw_h1, bw_h1], 2)

            with tf.variable_scope("lstm2"):
                cell_fw2 = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)
                cell_bw2 = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)

                d_cell_fw2 = tf.contrib.rnn.DropoutWrapper(cell_fw2, input_keep_prob=input_keep_prob)
                d_cell_bw2 = tf.contrib.rnn.DropoutWrapper(cell_bw2, input_keep_prob=input_keep_prob)

                (fw_h2, bw_h2), _ = tf.nn.bidirectional_dynamic_rnn(d_cell_fw2, d_cell_bw2, h1, sequence_length=x_len, dtype='float', scope='lstm2')
                h = tf.concat([fw_h2, bw_h2], 2)
            
            with tf.variable_scope("local_attention"):
                W_att_local = tf.get_variable("W_att_local", shape=[2*hidden_size, 1], initializer=tf.truncated_normal_initializer(mean=0.0, stddev=0.1, seed=gpu_idx*10))
                flat_h = tf.reshape(h, [batch_size*max_sent_size, tf.shape(h)[2]])
                h_tanh = tf.tanh(flat_h)
                u_flat = tf.matmul(h_tanh, W_att_local)
                u_local = tf.reshape(u_flat, [batch_size, max_sent_size])
                final_u = (tf.cast(x_mask[gpu_idx], 'float') -1)*VERY_BIG_NUMBER + u_local
                c_local = tf.map_fn(lambda i:tf.reduce_sum(tf.multiply(h[:, tf.maximum(0, i-width-1):tf.minimum(1+width+i, max_sent_size)],
                    tf.expand_dims(tf.nn.softmax(final_u[:, tf.maximum(0, i-width-1):tf.minimum(1+width+i, max_sent_size)], 1), 2)), axis=1),
                    tf.range(max_sent_size), dtype=tf.float32)  
                
            c_local = tf.transpose(c_local, perm=[1,0,2]) 
            h_final = tf.concat([tf.multiply(c_local, tf.expand_dims(float_x_mask, 2)), h], 2)
            flat_h_final = tf.reshape(h_final, [-1, tf.shape(h_final)[2]])
    
            
            with tf.variable_scope("hidden_layer"):
                W = tf.get_variable("W", shape=[4*hidden_size, 2*hidden_size], initializer=tf.truncated_normal_initializer(mean=0.0, stddev=0.1, seed=gpu_idx*20))
                b = tf.get_variable("b", shape=[2*hidden_size], initializer=tf.zeros_initializer())
                drop_flat_h_final = tf.nn.dropout(flat_h_final, input_keep_prob)
                flat_hl = tf.matmul(drop_flat_h_final, W) + b

            with tf.variable_scope("softmax_layer"):
                W = tf.get_variable("W", shape=[2*hidden_size, num_senses], initializer=tf.truncated_normal_initializer(mean=0.0, stddev=0.1, seed=gpu_idx*20))
                b = tf.get_variable("b", shape=[num_senses], initializer=tf.zeros_initializer())
                drop_flat_hl = tf.nn.dropout(flat_hl, input_keep_prob)
                flat_logits_sense = tf.matmul(drop_flat_hl, W) + b
                logits = tf.reshape(flat_logits_sense, [batch_size, max_sent_size, num_senses])
                predictions.append(tf.argmax(logits, 2))

            with tf.variable_scope("softmax_layer_pos"):
                W = tf.get_variable("W", shape=[2*hidden_size, num_pos], initializer=tf.truncated_normal_initializer(mean=0.0, stddev=0.1, seed=gpu_idx*30))
                b = tf.get_variable("b", shape=[num_pos], initializer=tf.zeros_initializer())
                flat_h1 = tf.reshape(h1, [-1, tf.shape(h1)[2]])
                drop_flat_hl = tf.nn.dropout(flat_hl, input_keep_prob)
                flat_logits_pos = tf.matmul(drop_flat_hl, W) + b
                logits_pos = tf.reshape(flat_logits_pos, [batch_size, max_sent_size, num_pos])
                log_likelihood, trans_params = tf.contrib.crf.crf_log_likelihood(logits_pos, y_pos[gpu_idx], x_len)
                loss_pos = crf_lambda*tf.reduce_mean(-log_likelihood)
                predictions_pos.append(logits_pos)
                total_trans_params.append(trans_params)

            float_sense_mask = tf.cast(sense_mask[gpu_idx], 'float')

            loss = tf.contrib.seq2seq.sequence_loss(logits, y[gpu_idx], float_sense_mask, name="loss")
            l2_loss = l2_lambda * tf.losses.get_regularization_loss()

            total_loss = tf.cond(pretrain, lambda:loss_pos, lambda:loss + loss_pos + l2_loss)

            summaries.append(tf.summary.scalar("loss_{}".format(gpu_idx), loss))
            summaries.append(tf.summary.scalar("loss_pos_{}".format(gpu_idx), loss_pos))
            summaries.append(tf.summary.scalar("total_loss_{}".format(gpu_idx), total_loss))


            optimizer = tf.train.AdamOptimizer(learning_rate)
            grads_vars = optimizer.compute_gradients(total_loss)

            clipped_grads = grads_vars
            if(clipping == True):
                clipped_grads = [(tf.clip_by_norm(grad, clip_norm), var) for grad, var in clipped_grads]

            tower_grads.append(clipped_grads)
            losses.append(total_loss)

with tf.device('/gpu:0'):
    tower_grads = average_gradients(tower_grads)
    losses = tf.add_n(losses)/len(losses)
    apply_grad_op = optimizer.apply_gradients(tower_grads, global_step=global_step)
    summaries.append(tf.summary.scalar('total_loss', losses))
    summaries.append(tf.summary.scalar('learning_rate', learning_rate))

    variable_averages = tf.train.ExponentialMovingAverage(moving_avg_deacy, global_step)
    variables_averages_op = variable_averages.apply(tf.trainable_variables())

    train_op = tf.group(apply_grad_op, variables_averages_op)
    saver = tf.train.Saver(tf.global_variables())
    summary = tf.summary.merge(summaries)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [5]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.allow_soft_placement = True
sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())                          # For initializing all the variables
summary_writer = tf.summary.FileWriter(log_dir, sess.graph)          # For writing Summaries

In [6]:
save_period = 100
log_period = 100

def model(xx, yy, yy_pos, mask, smask, train_cond=True, pretrain_cond=False):
    num_batches = int(len(xx)/(batch_size*num_gpus))
    _losses = 0
    temp_loss = 0
    preds_sense = []
    true_sense = []
    preds_pos = []
    true_pos = []
    
    for j in range(num_batches): 
        
        s = j * batch_size * num_gpus
        e = (j+1) * batch_size * num_gpus
        xx_re = xx[s:e].reshape([num_gpus, batch_size, -1])
        yy_re = yy[s:e].reshape([num_gpus, batch_size, -1])
        yy_pos_re = yy_pos[s:e].reshape([num_gpus, batch_size, -1])
        mask_re = mask[s:e].reshape([num_gpus, batch_size, -1])
        smask_re = smask[s:e].reshape([num_gpus, batch_size, -1])
 
        feed_dict = {x:xx_re, y:yy_re, y_pos:yy_pos_re, x_mask:mask_re, sense_mask:smask_re, pretrain:pretrain_cond, is_train:train_cond, input_keep_prob:keep_prob, word_emb_mat:word_embedding}
        
        if(train_cond==True):
            _, _loss, step, _summary = sess.run([train_op, losses, global_step, summary], feed_dict)
            summary_writer.add_summary(_summary, step)
            
            temp_loss += _loss
            if((j+1)%log_period==0):
                print("Steps: {}".format(step), "Loss:{0:.4f}".format(temp_loss/log_period), ", Current Loss: {0:.4f}".format(_loss))
                temp_loss = 0
            if((j+1)%save_period==0):
                saver.save(sess, save_path=save_dir)                         
                
        else:
            _loss, pred, crf_logits, trans_params_ = sess.run([total_loss, predictions, predictions_pos, total_trans_params], feed_dict)
            
            for i in range(num_gpus):
                preds_sense.append(pred[i][smask_re[i]])
                true_sense.append(yy_re[i][smask_re[i]])
                true_pos.append(yy_pos_re[i][mask_re[i]])
                temp = []
                for k in range(batch_size):
                    logit_ = crf_logits[i][k][:sum(mask_re[i][k])] # keep only the valid steps
                    viterbi_seq, viterbi_score = tf.contrib.crf.viterbi_decode(logit_, trans_params_[i])
                    temp += viterbi_seq
                preds_pos.append(temp)

        _losses +=_loss

    if(train_cond==False): 
        sense_preds = []
        sense_true = []
        pos_preds = []
        pos_true = []
        
        for preds in preds_sense:
            for ps in preds:      
                sense_preds.append(ps)  
        for trues in true_sense:
            for ts in trues:
                sense_true.append(ts)
        
        for preds in preds_pos:
            for ps in preds:      
                pos_preds.append(ps)      
        for trues in true_pos:
            for ts in trues:
                pos_true.append(ts)
                
        return _losses/num_batches, sense_preds, sense_true, pos_preds, pos_true

    return _losses/num_batches, step

def eval_score(yy, pred, yy_pos, pred_pos):
    f1 = f1_score(yy, pred, average='macro')
    accu = accuracy_score(yy, pred)
    f1_pos = f1_score(yy_pos, pred_pos, average='macro')
    accu_pos = accuracy_score(yy_pos, pred_pos)
    return f1*100, accu*100, f1_pos*100, accu_pos*100

In [7]:
x_id_train = train_data['x']
mask_train = train_data['x_mask']
sense_mask_train = train_data['sense_mask']
y_train = train_data['y']
y_pos_train = train_data['pos']

x_id_val = val_data['x']
mask_val = val_data['x_mask']
sense_mask_val = val_data['sense_mask']
y_val = val_data['y']
y_pos_val = val_data['pos']

In [8]:
def testing():
    start_time = time.time()
    val_loss, val_pred, val_true, val_pred_pos, val_true_pos = model(x_id_val, y_val, y_pos_val, mask_val, sense_mask_val, train_cond=False)        
    f1_, accu_, f1_pos_, accu_pos_ = eval_score(val_true, val_pred, val_true_pos, val_pred_pos)
    time_taken = time.time() - start_time
    print("Val: F1 Score:{0:.2f}".format(f1_), "Accuracy:{0:.2f}".format(accu_), " POS: F1 Score:{0:.2f}".format(f1_pos_), "Accuracy:{0:.2f}".format(accu_pos_), "Loss:{0:.4f}".format(val_loss), ", Time: {0:.1f}".format(time_taken))
    return f1_, accu_, f1_pos_, accu_pos_

def training(current_epoch, pre_train_cond):
        random = np.random.choice(len(y_train), size=(len(y_train)), replace=False)
        x_id_train_tmp = x_id_train[random]
        y_train_tmp = y_train[random]
        mask_train_tmp = mask_train[random]    
        sense_mask_train_tmp = sense_mask_train[random]
        y_pos_train_tmp = y_pos_train[random]

        start_time = time.time()
        train_loss, step = model(x_id_train_tmp, y_train_tmp, y_pos_train_tmp, mask_train_tmp, sense_mask_train_tmp, pretrain_cond=pre_train_cond)
        time_taken = time.time() - start_time
        print("Epoch: {}".format(current_epoch+1),", Step: {}".format(step), ", loss: {0:.4f}".format(train_loss), ", Time: {0:.1f}".format(time_taken))
        saver.save(sess, save_path=save_dir)                         
        print("Model Saved")
        return [step, train_loss]

In [ ]:
loss_collection = []
val_collection = []
num_epochs = 20
val_period = 2

# Pretraining POS Tags
training(0, True)
training(1, True)
testing()

for i in range(num_epochs):
    loss_collection.append(training(i, False))
    if((i+1)%val_period==0):
        val_collection.append(testing())

Steps: 100 Loss:2.7530 , Current Loss: 1.3150
Steps: 200 Loss:1.0997 , Current Loss: 0.8834
Steps: 300 Loss:0.8495 , Current Loss: 0.7899
Steps: 400 Loss:0.7236 , Current Loss: 0.6715
Steps: 500 Loss:0.6522 , Current Loss: 0.6031
Steps: 600 Loss:0.6055 , Current Loss: 0.6011
Steps: 700 Loss:0.5653 , Current Loss: 0.5640
Steps: 800 Loss:0.5363 , Current Loss: 0.5527
Epoch: 1 , Step: 885 , loss: 0.9292 , Time: 6593.3
Model Saved
Steps: 985 Loss:0.4982 , Current Loss: 0.5113
Steps: 1085 Loss:0.4793 , Current Loss: 0.4804
Steps: 1185 Loss:0.4628 , Current Loss: 0.4392
Steps: 1285 Loss:0.4512 , Current Loss: 0.4618
Steps: 1385 Loss:0.4417 , Current Loss: 0.4541
Steps: 1485 Loss:0.4313 , Current Loss: 0.4190
Steps: 1585 Loss:0.4195 , Current Loss: 0.4209
Steps: 1685 Loss:0.4135 , Current Loss: 0.4002
Epoch: 2 , Step: 1770 , loss: 0.4453 , Time: 6481.8
Model Saved


/users/btech/aviraj/envs/lib/python3.5/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Val: F1 Score:1.30 Accuracy:1.98  POS: F1 Score:87.26 Accuracy:92.35 Loss:6.5045 , Time: 1338.7
Steps: 1870 Loss:3.0857 , Current Loss: 2.7427
Steps: 1970 Loss:2.7031 , Current Loss: 2.5829
Steps: 2070 Loss:2.6134 , Current Loss: 2.5436
Steps: 2170 Loss:2.5458 , Current Loss: 2.5251
Steps: 2270 Loss:2.5081 , Current Loss: 2.6032
Steps: 2370 Loss:2.4721 , Current Loss: 2.3927
Steps: 2470 Loss:2.4513 , Current Loss: 2.4687
Steps: 2570 Loss:2.4191 , Current Loss: 2.4182
Epoch: 1 , Step: 2655 , loss: 2.5810 , Time: 6424.4
Model Saved
Steps: 2755 Loss:2.3788 , Current Loss: 2.3777
Steps: 2855 Loss:2.3645 , Current Loss: 2.3862
Steps: 2955 Loss:2.3436 , Current Loss: 2.3440
Steps: 3055 Loss:2.3291 , Current Loss: 2.3917
Steps: 3155 Loss:2.3128 , Current Loss: 2.3305
Steps: 3255 Loss:2.3004 , Current Loss: 2.3307
Steps: 3355 Loss:2.2989 , Current Loss: 2.2376
Steps: 3455 Loss:2.2747 , Current Loss: 2.3122
Epoch: 2 , Step: 3540 , loss: 2.3196 , Time: 6384.8
Model Saved


/users/btech/aviraj/envs/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Val: F1 Score:30.57 Accuracy:43.43  POS: F1 Score:84.94 Accuracy:91.65 Loss:2.2736 , Time: 1334.1
Steps: 3640 Loss:2.2597 , Current Loss: 2.2908
Steps: 3740 Loss:2.2459 , Current Loss: 2.2992
Steps: 3940 Loss:2.2260 , Current Loss: 2.2485
Steps: 4040 Loss:2.2197 , Current Loss: 2.1538
Steps: 4140 Loss:2.2151 , Current Loss: 2.2150
Steps: 4240 Loss:2.2052 , Current Loss: 2.2117
Steps: 4340 Loss:2.1925 , Current Loss: 2.2674
Epoch: 3 , Step: 4425 , loss: 2.2232 , Time: 6403.8
Model Saved
Steps: 4525 Loss:2.1854 , Current Loss: 2.1272
Steps: 4625 Loss:2.1682 , Current Loss: 2.2396
Steps: 4725 Loss:2.1652 , Current Loss: 2.1550
Steps: 4825 Loss:2.1640 , Current Loss: 2.1435
Steps: 4925 Loss:2.1604 , Current Loss: 2.0998
Steps: 5025 Loss:2.1477 , Current Loss: 2.1546
Steps: 5125 Loss:2.1531 , Current Loss: 2.1015
Steps: 5225 Loss:2.1476 , Current Loss: 2.1499
Epoch: 4 , Step: 5310 , loss: 2.1595 , Time: 6392.6
Model Saved
Val: F1 Score:34.55 Accuracy:46.59  POS: F1 Score:85.45 Accuracy:92.0

In [ ]:
loss_collection = []
val_collection = []
num_epochs = 20
val_period = 2

for i in range(num_epochs):
    loss_collection.append(training(i, False))
    if((i+1)%val_period==0):
        val_collection.append(testing())

Steps: 11920 Loss:1.9396 , Current Loss: 1.9192
Steps: 12020 Loss:1.9274 , Current Loss: 2.0037
Steps: 12120 Loss:1.9368 , Current Loss: 1.9705
Steps: 12220 Loss:1.9159 , Current Loss: 1.9314
Steps: 12320 Loss:1.9359 , Current Loss: 1.8787
Steps: 12420 Loss:1.9194 , Current Loss: 1.9753
Steps: 12520 Loss:1.9213 , Current Loss: 1.9960
Steps: 12620 Loss:1.9280 , Current Loss: 1.8762
Epoch: 1 , Step: 12705 , loss: 1.9274 , Time: 8243.8
Model Saved
Steps: 12805 Loss:1.9194 , Current Loss: 1.9295
Steps: 12905 Loss:1.9119 , Current Loss: 1.9011
Steps: 13005 Loss:1.9154 , Current Loss: 1.8299
Steps: 13105 Loss:1.9119 , Current Loss: 1.8779
Steps: 13205 Loss:1.9129 , Current Loss: 1.8713
Steps: 13305 Loss:1.9084 , Current Loss: 1.9974
Steps: 13405 Loss:1.9120 , Current Loss: 1.9565
Steps: 13505 Loss:1.9055 , Current Loss: 1.8886
Epoch: 2 , Step: 13590 , loss: 1.9118 , Time: 9350.0
Model Saved
Val: F1 Score:43.30 Accuracy:51.93  POS: F1 Score:86.61 Accuracy:92.86 Loss:1.9281 , Time: 2102.4
Step

In [10]:
loss_collection = []
val_collection = []
num_epochs = 20
val_period = 2

for i in range(num_epochs):
    loss_collection.append(training(i, False))
    if((i+1)%val_period==0):
        val_collection.append(testing())

Steps: 14590 Loss:1.8898 , Current Loss: 1.8106
Steps: 14690 Loss:1.8922 , Current Loss: 1.9134
Steps: 14790 Loss:1.8830 , Current Loss: 1.9143
Steps: 14890 Loss:1.8821 , Current Loss: 1.9095
Steps: 14990 Loss:1.8854 , Current Loss: 1.8719
Steps: 15090 Loss:1.8841 , Current Loss: 1.9275
Steps: 15190 Loss:1.8855 , Current Loss: 1.9128
Steps: 15290 Loss:1.8766 , Current Loss: 1.9220
Epoch: 1 , Step: 15375 , loss: 1.8842 , Time: 10384.0
Model Saved
Steps: 15475 Loss:1.8735 , Current Loss: 1.7995
Steps: 15575 Loss:1.8888 , Current Loss: 1.7857
Steps: 15675 Loss:1.8689 , Current Loss: 1.8067
Steps: 15775 Loss:1.8728 , Current Loss: 1.8233
Steps: 15875 Loss:1.8789 , Current Loss: 1.9815
Steps: 15975 Loss:1.8744 , Current Loss: 1.8223
Steps: 16075 Loss:1.8705 , Current Loss: 1.8704
Steps: 16175 Loss:1.8795 , Current Loss: 1.8332
Epoch: 2 , Step: 16260 , loss: 1.8753 , Time: 8197.5
Model Saved
Val: F1 Score:43.75 Accuracy:52.86  POS: F1 Score:86.77 Accuracy:92.98 Loss:1.8804 , Time: 1568.8
Ste

In [10]:
loss_collection = []
val_collection = []
num_epochs = 20
val_period = 2

for i in range(num_epochs):
    loss_collection.append(training(i, False))
    if((i+1)%val_period==0):
        val_collection.append(testing())

Steps: 32790 Loss:1.7450 , Current Loss: 1.7841
Steps: 32890 Loss:1.7457 , Current Loss: 1.6578
Steps: 32990 Loss:1.7445 , Current Loss: 1.7572
Steps: 33090 Loss:1.7359 , Current Loss: 1.7636
Steps: 33190 Loss:1.7379 , Current Loss: 1.7785
Steps: 33290 Loss:1.7408 , Current Loss: 1.8216
Steps: 33390 Loss:1.7497 , Current Loss: 1.7257
Steps: 33490 Loss:1.7443 , Current Loss: 1.8135
Epoch: 1 , Step: 33575 , loss: 1.7427 , Time: 10418.4
Model Saved
Steps: 33675 Loss:1.7355 , Current Loss: 1.6992
Steps: 33775 Loss:1.7324 , Current Loss: 1.7632
Steps: 33875 Loss:1.7411 , Current Loss: 1.6326
Steps: 33975 Loss:1.7422 , Current Loss: 1.7930
Steps: 34075 Loss:1.7450 , Current Loss: 1.7360
Steps: 34175 Loss:1.7369 , Current Loss: 1.7717
Steps: 34275 Loss:1.7361 , Current Loss: 1.7250
Steps: 34375 Loss:1.7365 , Current Loss: 1.7765
Epoch: 2 , Step: 34460 , loss: 1.7374 , Time: 7592.0
Model Saved
Val: F1 Score:48.27 Accuracy:55.87  POS: F1 Score:87.41 Accuracy:93.47 Loss:1.7595 , Time: 1358.4
Ste

In [ ]:
start_time = time.time()
train_loss, train_pred, train_true, train_pred_pos, train_true_pos = model(x_id_train, y_train, y_pos_train, mask_train, sense_mask_train, train_cond=False)        
f1_, accu_, f1_pos_, accu_pos_ = etrain_score(train_true, train_pred, train_true_pos, train_pred_pos)
time_taken = time.time() - start_time
print("train: F1 Score:{0:.2f}".format(f1_), "Accuracy:{0:.2f}".format(accu_), " POS: F1 Score:{0:.2f}".format(f1_pos_), "Accuracy:{0:.2f}".format(accu_pos_), "Loss:{0:.4f}".format(train_loss), ", Time: {0:.1f}".format(time_taken))

In [9]:
saver.restore(sess, save_dir)